In [ ]:
#pip install folium
#!pip install geopandas
#!pip install geojson

In [ ]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import geopandas as gpd
from shapely.geometry import Point, Polygon
import geojson

In [ ]:
with open("../datasets/data.pkl", "rb") as file1:
    data = pickle.load(file1)

Listando las primeras 10 filas del DataFrame

In [ ]:
data.head(10)

Consultando información sobre tipo de índice, los tipos de datos por columna, valores no nulos y uso de memoria en el DataFrame


In [ ]:
data.info()

Consulta del resumen estadístico de las columnas numéricas presentes en el DataFrame. Este método calcula algunas medidas estadísticas como percentil, media y desviación estándar de los valores numéricos del DataFrame

In [ ]:
data.describe()

Consultando el Porcentaje de valores nulos por columna

In [ ]:
porcentaje_nulos = data.isna().sum().sort_values(ascending=False) / len(data)
porcentaje_nulos

In [ ]:
porcentaje_nulos[porcentaje_nulos != 0].plot(kind='barh')

El procentaje de valores nulos por Columna, es muy bajo y no llegan al 30% lo cual es una buena señal para no prescindir de esas filas o rellenarlos a través de la media

In [ ]:
print(f"Numero de filas duplicadas = {data.duplicated().sum()}")

Realizamos una descripción estadística sobre las de columnas de tipo texto

In [ ]:

tipo_texto = data.select_dtypes(include=['object']).columns
df_texto = data[tipo_texto]
df_texto.describe()

Revisión de Valores Atípicos/Extremos u Outliers

In [ ]:
df_numerico = data.select_dtypes(include=['number']) 

In [ ]:
df_numerico

In [ ]:
Q1 = df_numerico.quantile(0.25)
Q3 = df_numerico.quantile(0.75)
IQR = Q3 - Q1 
limite_inf = Q1 - 1.5 * IQR
limite_sup = Q3 + 1.5 * IQR
mascaras = (df_numerico < limite_inf) | (df_numerico > limite_sup)
valores_atipicos = df_numerico[mascaras]

valores_atipicos.nro_victimas[valores_atipicos.nro_victimas.notnull()].head(10)

Los outliers para la columna "nro_victimas" son los valores de 2 o 3 porque la mayor cantidad de registros estan asociados a tener como número de victimas el valor de 1

### Matriz de Correlación

In [ ]:
data

In [ ]:
# Generamos un mapa de calor con las correlaciones de los datos
dataNumerica = data.select_dtypes(include=['number']) 
corr = dataNumerica.corr().round(2)
plt.figure(figsize=(6,5))
sns.heatmap(corr, cmap="YlGnBu", annot=True)
plt.show()

Se puede verificar que existe una correlación entre la Edad y la Franja Horaria, en un posterior Análisis en PowerBi vimos la intersección con el sexo y se obtuvo una valiosa información

## Análisis exploratorio y visualización

Columnas que analizaremos:

1. Comuna
2. Franja horaria
3. Latitud, Longitud

A continuación se listan las comunas que existen en la Ciudad Autónoma de Buenos Aires(CABA)

In [ ]:
data.comuna.unique()

Listado de Comunas respecto a la Cantidad de Siniestros Viales en la Ciudad Autónoma de Buenos Aires(CABA)

In [ ]:
comunas_by_accident = data.comuna.value_counts().sort_values()
comunas_by_accident

In [ ]:
comunas_by_accident[:17].plot(kind='barh')

Considerando un Top 3 de las principales Comunas asociadas a una mayor cantidad de Siniestros Viales, son : Comuna 1, Comuna 4 y Comuna 9

Valores que contiene la columna "Franja Horaria" vital para un posterior análisis sobre el rango de horas mas frecuentes en Siniestros Viales

In [ ]:
data.franja_horaria.unique()

In [ ]:
franja_accidente = data.franja_horaria.value_counts().sort_values()
franja_accidente

In [ ]:
franja_accidente[:25].plot(kind='barh')

In [ ]:
dataHisto=data[['franja_horaria','nro_victimas','edad']]
dataHisto.hist(bins = 50, figsize= (12,10))
plt.show()

Por medio de los gráficos anteriores podemos identificar el rango de edad de 20 a 40 años y la Franja Horaria de 5 a 10 estan asociados a una mayor cantidad de Siniestros Viales 

Se puede reconocer que el rango de horas que posee más Accidentes de Tránsitos está entre las 5 - 7 horas de la mañana

In [ ]:
title_words = ' '.join(data['direc_normalizada'].astype(str))
title_words = title_words.lower()
stopwords = ["de", "y","el", "la", "&", ":", "para", "por", "en",  "con"]
wordcloud = WordCloud(stopwords=stopwords, background_color='white').generate(title_words)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de Palabras - Dirección Normalizada')
plt.show()

Al realizar el gráfico de nube de palabras de la columna "Dirección Normalizada" se puede inferir que tipo el de vías asociados a una mayor cantidad de Siniestros Viales es la Avenida aquí podemos reconocer a la Avenida Gral Paz en un analisis posterior será importante identificar las avenidas por comuna que presentan mayor cantidad de Accidentes de Tránsito

Consultando los valores de Latitud y longitud para un posterior Análisis de GeoLocalización

In [ ]:
data.latitud

In [ ]:
data.longitud

Hallamos la localización de los Siniestros Viales a partir de la latitud y longitud


In [ ]:
dataLocalizacion = data[data['latitud'].notnull()]
dataLocalizacion = dataLocalizacion[dataLocalizacion['longitud'].notnull()]

lat_lon_pairs = list(zip(list(dataLocalizacion.latitud), list(dataLocalizacion.longitud)))

Usando Folium para generar un mapa simple a partir de los datos de Localización

In [ ]:
map = folium.Map(location=[-58,-34], zoom_start=3.4)
HeatMap(lat_lon_pairs).add_to(map)


In [ ]:
map

Antes de comenzar a usar geopandas, se define el sistema de referencia de coordenadas (CRS) este le dice a Python cómo se relacionan esas coordenadas con los lugares de la Tierra luego se necesita decirle a Python el sistema de referencia de coordenadas.

In [ ]:

crs={'init':'epsg:4326'}


Se define la geometría

In [ ]:

geometry=[Point(xy) for xy in zip(data["longitud"], data["latitud"])]

Se tiene sistema de referencia y la geometría en su lugar, finalmente carguemos nuestros datos

In [ ]:

geodata=gpd.GeoDataFrame(data,crs=crs, geometry=geometry)

Se muestran los puntos de datos que se asignaron en función de la latitud y la longitud 

In [ ]:

geodata.plot()

Ahora se mapearan los puntos en un mapa real y se necesita usar un archivo Shapefile de la Ciudad de Buenos Aires

In [ ]:

shapefile = gpd.read_file(r'../comunas/comunas_wgs84.shp')

Ahora podemos trazar el mapa de la Ciudad Atónoma de Buenos Aires y luego agregar los puntos que hacen referencia a la localización de los Siniestros Viales

In [ ]:

fig, ax = plt.subplots(figsize=(7,7))

shapefile.plot(ax=ax, facecolor='Grey', edgecolor='k',alpha=1,linewidth=1,cmap="cividis")

geodata.plot(ax=ax, color='red', markersize=5);

fig.suptitle('Ciudad Buenos Aires', fontsize=12)
ax.set_xlabel('Longitud', fontsize=10)
ax.set_ylabel('Latitud', fontsize='medium')